In [1]:
#--------------------------- Config.py ---------------------------#
class Config:
    TRAINSET_PATH = 'Data/Training2/'
    MODEL_PATH = 'LSTMAutoEncoderModel'#"model" + str(1)
    BATCH_SIZE = 3
    EPOCHS = 1000

In [2]:
#-------------------------- Imports --------------------------#
from os import listdir
from os.path import isfile, join, isdir
from PIL import Image
import numpy as np
import shelve
import cv2
from tensorflow.keras.layers import Conv2DTranspose, ConvLSTM2D, BatchNormalization, TimeDistributed, Conv2D, LayerNormalization
from tensorflow.keras.models import Sequential, load_model
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import tensorflow
import sys
import Hyperparameters as hyp

lr_opts = [0.0001, 0.001]
decay_opts = [0.00001, 0.0001]
eps_opts = [0.000001, 0.00001]
num_filters_opts = [128, 64]
kernel_size_opts = [11, 13]

In [3]:
#--------------------------- AutoencoderParsing.py ---------------------------#
def get_sliding_windows(frames_list, sequence_size):
    """ For data augmenting purposes.
    Parameters
    ----------
    frames_list : list
        A list of sorted frames of shape hyp.FRAME_RES X hyp.FRAME_RES
    sequence_size: int
        The size of the lstm sequence
    Returns
    -------
    list
        A list of clips , 20 frames each
    """
    clips = []
 
    for i in range(len(frames_list) - sequence_size):
        clip = np.zeros(shape=(sequence_size, hyp.FRAME_RES, hyp.FRAME_RES, 1))
        clip[:, :, :, 0] = frames_list[i : i + sequence_size]
        clips.append(clip)
    return np.array(clips)

def getData(path):
    data = []
    for file in listdir(path):
        if file[-3:] == "npy":
            all_frames = np.load(path + file, allow_pickle=True)
            data.extend(get_sliding_windows(frames_list=all_frames, sequence_size=hyp.SEQ_LEN))
    return np.array(data)

In [4]:
#--------------------------- LSTM_Autoencoder_Model.py ---------------------------#
def create_model(num_filters, kernel_size):
    seq = Sequential()
    seq.add(TimeDistributed(Conv2D(int(num_filters/2), kernel_size, strides=4, padding="same"), batch_input_shape=(None, hyp.CLIP_LEN, hyp.FRAME_RES, hyp.FRAME_RES, 1)))
    seq.add(LayerNormalization())
    seq.add(TimeDistributed(Conv2D(int(num_filters/4), kernel_size - 6, strides=2, padding="same")))
    seq.add(LayerNormalization())
    # # # # #
    seq.add(ConvLSTM2D(int(num_filters/4), kernel_size - 8, padding="same", return_sequences=True))
    seq.add(LayerNormalization())
    seq.add(ConvLSTM2D(int(num_filters/8), kernel_size - 8, padding="same", return_sequences=True))
    seq.add(LayerNormalization())
    seq.add(ConvLSTM2D(int(num_filters/4), kernel_size - 8, padding="same", return_sequences=True))
    seq.add(LayerNormalization())
    # # # # #
    seq.add(TimeDistributed(Conv2DTranspose(int(num_filters/4), kernel_size - 6, strides=2, padding="same")))
    seq.add(LayerNormalization())
    seq.add(TimeDistributed(Conv2DTranspose(int(num_filters/2), kernel_size, strides=4, padding="same")))
    seq.add(LayerNormalization())
    seq.add(TimeDistributed(Conv2D(1, kernel_size, activation="sigmoid", padding="same")))
#     print(seq.summary())
    return seq

In [ ]:
#--------------------------- TrainAutoencoder.py ---------------------------#
# training_set = ap.get_training_set()
training_set = np.array(getData(Config.TRAINSET_PATH), copy=False)
print("Shape of training_set before split: ", training_set.shape)
training_set, validation_set, _, _ = train_test_split(training_set, training_set, test_size=0.1)
print("Shape of training_set: ", training_set.shape)
print("Shape of validation_set: ", validation_set.shape)
best_lr, best_decay, best_eps, best_filters, best_kernels = 0, 0, 0, 0, 0
val_loss = sys.maxsize
for lr in lr_opts:
    for decay in decay_opts:
        for eps in eps_opts:
            for num_filters in num_filters_opts:
                for kernel_size in kernel_size_opts:
                    model = create_model(num_filters, kernel_size)
                    model.compile(loss='mse', optimizer=tensorflow.keras.optimizers.Adam(lr=lr, decay=decay, epsilon=eps))
                    es = EarlyStopping(min_delta=eps, verbose=1, patience=3)
                    history = model.fit(training_set, training_set, batch_size=Config.BATCH_SIZE, epochs=Config.EPOCHS,
                                        validation_data=(validation_set, validation_set), callbacks=[es])
                    final_val_loss = history.history['val_loss'].pop()
                    print("val_loss: ", final_val_loss)
                    print(f"lr = {lr}")
                    print(f"decay = {decay}")
                    print(f"eps = {eps}")
                    print(f"num_filters = {num_filters}")
                    print(f"kernel_size = {kernel_size}")
                    if final_val_loss < val_loss:
                        val_loss = final_val_loss
                        best_lr = lr
                        best_decay = decay
                        best_eps = eps
                        best_filters = num_filters
                        best_kernels = kernel_size
                        model.save("LSTM_Auto")
print('The optimal training hyperparams are:')
print(f"lr = {best_lr}")
print(f"decay = {best_decay}")
print(f"eps = {best_eps}")
print(f"num_filters = {best_filters}")
print(f"kernel_size = {best_kernels}")
print(f"Best val_loss = {val_loss}")

Shape of training_set before split:  (45093, 10, 32, 32, 1)
Shape of training_set:  (40583, 10, 32, 32, 1)
Shape of validation_set:  (4510, 10, 32, 32, 1)
Epoch 1/1000
13528/13528 [==============================] - 3069s 227ms/step - loss: 13804.5645 - val_loss: 13786.9248
Epoch 2/1000
 3232/13528 [======>.......................] - ETA: 38:21 - loss: 13820.0391